In [1]:
%pip install scrapy

  Using cached pyOpenSSL-25.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ----------------------------- ---------- 2.4/3.2 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 11.1 MB/s eta 0:00:00
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------------ --------------- 2.4/3.8 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 10.8 MB/s eta 0:00:00
Using cached pyOpenSSL-25.0.0-py3-none-any.whl (56 kB)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ------------- -------------------------- 1.0/3.2 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 9.9 MB/s eta 0:00:00
Using cached pyasn1-

In [1]:
from pathlib import Path
import scrapy
import pandas as pd
import re
import os


class NewsSpider(scrapy.Spider):
    name = "news"
    custom_settings = {
        'JOBDIR': 'crawls/news_spider_state',
        # 'FEED': {} # Scrapy 기본 저장 방식 비활성화
    }

    # 합친 csv에서 url 목록 불러오기
    def start_requests(self):
        df = pd.read_csv('C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/Financial_News_text')
        urls = df['url'].tolist()

        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    # 웹에서 본문 크롤링
    def parse(self, response):
        title = response.css('h2#title_area span::text').get(default='N/A')
        date_raw = response.css('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME::attr(data-date-time)').get(default="N/A")
        content = ' '.join(response.css('div.newsct_article p::text').getall()).strip()

        # 로그에 date_raw 값 출력 (디버깅용)
        self.log(f"DEBUG: Extracted date_raw: {date_raw}")

        # 날짜에서 시간 제거
        if date_raw and date_raw != 'N/A':
            clean_date = date_raw.split(' ')[0]
        else:
            clean_date = 'unknown_date'

        # URL에서 고유 기사 ID 추출
        article_id_match = re.search(r'/(\d{10})\?', response.url)
        article_id = article_id_match.group(1) if article_id_match else "no_id"

        # 텍스트파일 저장할 폴더 생성 (없으면 자동 생성)
        save_path = "news_texts"
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # 파일명 만들기
        file_title = re.sub(r'[\\/*?:"<>|]', "_", clean_date)
        file_name = f'news_{file_title}_{article_id}.txt'

        with open(f'news_texts/{file_name}','w', encoding='utf-8') as f:
            f.write(f'{title}\n')
            f.write(f'{clean_date}\n')
            f.write(f'{content}\n')

        self.log(f'Saved file: {file_name}')